In [22]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score


import sys
import re
import emoji
import contractions
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import unicodedata

# Load the CSV file
data = pd.read_csv('train_val.csv')

# Define the list of concern labels
concern_labels = ['unnecessary', 'mandatory', 'pharma', 'conspiracy', 'political', 'country',
                  'rushed', 'ingredients', 'side-effect', 'ineffective', 'religious', 'none']

# Split data into input (tweets) and labels
tweets = data['tweet'].tolist()

# Create label columns
for label in concern_labels:
    data[label] = data['labels'].apply(lambda x: int(label in x.split()))
    data[label] = data['labels'].apply(lambda x: int(label in x.split()))

labels = data[['unnecessary', 'mandatory', 'pharma', 'conspiracy', 'political', 'country',
               'rushed', 'ingredients', 'side-effect', 'ineffective', 'religious', 'none']].values.tolist()


data = data.drop(['labels'], axis=1)
data.head(5)

[nltk_data] Downloading package stopwords to /home/dell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,ID,tweet,unnecessary,mandatory,pharma,conspiracy,political,country,rushed,ingredients,side-effect,ineffective,religious,none
0,1296010336907038720t,@cath__kath AstraZeneca is made with the kidne...,0,0,0,0,0,0,0,1,0,0,0,0
1,1336808189677940736t,It begins. Please find safe alternatives to th...,0,0,0,0,0,0,0,0,1,0,0,0
2,1329488407307956231t,"@PaolaQP1231 Well, I mean congratulations Covi...",0,0,0,0,0,0,0,0,1,0,0,0
3,1364194604459900934t,@BorisJohnson for those of us that do not wish...,0,1,0,0,0,0,0,0,0,0,0,0
4,1375938799247765515t,She has been trying to speak out: writing lett...,0,0,0,0,0,0,1,0,1,0,0,0


In [23]:
def cleaning(text):
  text= text.lower()
  text= emoji.demojize(text)
  text=contractions.fix(text)
  text=text.strip()
  text=text.replace('[^\w\s]','')
  text=re.sub(r'http\S+', '', text)
  text = ' '.join(word for word in text.split(' ') if not word.startswith('@'))
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
  text = REPLACE_BY_SPACE_RE.sub(' ' , text)
  text = BAD_SYMBOLS_RE.sub(' ',text)
  
  return text
data['tweet'] = data['tweet'].apply(cleaning)
data['tweet'].iloc[0]

'astrazeneca is made with the kidney cells of a little girl aborted back in the 70s '

In [24]:
data['ID'] = data['ID'].astype(str)
data['tweet'] = data['tweet'].astype(str)

In [27]:
data.describe()

,unnecessary,mandatory,pharma,conspiracy,political,country,rushed,ingredients,side-effect,ineffective,religious,none
count,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.000000,9921.00000,9921.000000,9921.000000,9921.000000
mean,0.072775,0.078923,0.128314,0.049088,0.063098,0.020260,0.148775,0.043947,0.38353,0.168531,0.006451,0.063401
std,0.259780,0.269633,0.334456,0.216062,0.243152,0.140896,0.355885,0.204988,0.48627,0.374356,0.080062,0.243695
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [28]:
# Split the data into train and test sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
# test_data, val_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [29]:
# saving the dataframe
train_data.to_csv('train_data.csv',index=False)
val_data.to_csv('val_data.csv',index=False)
# test_data.to_csv('test_data.csv',index=False)

In [42]:
data = pd.read_csv('test.csv')
data['tweet'] = data['tweet'].apply(cleaning)
print(data['tweet'].iloc[0])
data['id'] = data['id'].astype(str)
data['tweet'] = data['tweet'].astype(str)
# data = data.drop(['id'], axis=1)

study links hpv vaccine to historically high infertility rates   spirituality  self


In [43]:
data

,id,tweet
0,1070378532260470789t,study links hpv vaccine to historically high i...
1,973746711964372993t,deaths from tainted measles vaccine affecting ...
2,1043031076787040257t,am apreciat un videoclip pe treatment and cur...
3,1066338147527741440t,video gt gt mmr vaccine increase risk of...
4,963522018544152576t,oral polio vaccine infecting unvaccinated kid...
...,...,...
481,1099937642169405440t,i know it is hard to believe but another va...
482,1536674508731518985t,seizures on the very day of the mmr vaccine t...
483,1183314494874968064t,kenyan doctors say unicef is making women barr...
484,1327581896243556352t,not doing it my experience with vaccines in ...


In [44]:
data.to_csv('test_data.csv',index=False)